In [1]:
import torch
import logging
import torch.nn as nn
from tqdm import tqdm
from pathlib import Path
from torch import autograd
from sklearn import model_selection
from torch.utils.data import Dataset, Subset, DataLoader

from strokes import StrokePatientsMIDataset
from strokesdict import STROKEPATIENTSMI_LOCATION_DICT
import scipy
from torcheeg.transforms import Select,BandSignal,Compose
from to import ToGrid, ToTensor
from typing import Callable, Dict, Union, List
import numpy as np
import soxr
from downsample import SetSamplingRate
from baseline import BaselineCorrection
from torcheeg.transforms import EEGTransform, Select,BandSignal,Compose

dataset = StrokePatientsMIDataset(root_path='../../mi_swin/subdataset',
                                  io_path='.torcheeg/datasets_1739513177339_4Vlqh',
                        chunk_size=500,  # 1 second
                        overlap = 0,
                        offline_transform=Compose(
                                [BaselineCorrection(),
                                SetSamplingRate(origin_sampling_rate=500,target_sampling_rate=128),
                                BandSignal(sampling_rate=128,band_dict={'frequency_range':[8,40]})
                                ]),
                        online_transform=Compose(
                                # [ToTensor()]),
                                [ToGrid(STROKEPATIENTSMI_LOCATION_DICT),ToTensor()]),
                
                        label_transform=Select('label'),
                        num_worker=8
)
print(dataset[0][0].shape) #EEG shape(1,30,128)
print(dataset[0][1])  # label (int)
print(len(dataset))

/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2025-02-14 14:08:11] INFO (torcheeg/MainThread) 🔍 | Detected cached processing results, reading cache from .torcheeg/datasets_1739513177339_4Vlqh.


torch.Size([1, 128, 9, 9])
0
320


In [2]:
import os
import shutil

def delete_folder_if_exists(target_folder_name):
    # 获取父文件夹中的所有内容
    parent_folder = os.getcwd()
    for folder_name in os.listdir(parent_folder):
        folder_path = os.path.join(parent_folder, folder_name)

        # 检查是否是文件夹并且名称是否匹配
        if os.path.isdir(folder_path) and folder_name == target_folder_name:
            try:
                # 删除目标文件夹
                shutil.rmtree(folder_path)
                print(f"已删除文件夹: {folder_path}")
            except Exception as e:
                print(f"删除文件夹 {folder_path} 时出错: {e}")

In [8]:
from swin_transformer3D import SwinTransformer3D
import numpy as np
import torch

input = torch.randn(5,1,128,9,9)

model = SwinTransformer3D(patch_size=(4,3,3),
                          in_chans=1,
                          depths=[2,2,4],
                          num_heads=[3,3,6],
                          window_size=(2,2,2),
                          mlp_ratio=2.,
                          drop_rate=0.1,
                          attn_drop_rate=0.1,
                          drop_path_rate=0.3,
                          num_classes=2
                          )

output = model(input)

print(output.shape)

torch.Size([5, 96, 32, 3, 3])
torch.Size([5, 192, 8, 3, 3])
torch.Size([5, 384, 2, 3, 3])
torch.Size([5, 384, 2, 3, 3])
torch.Size([5, 2])


In [6]:
from swin_transformer3D import SwinTransformer3D

HYPERPARAMETERS = {
    "seed": 12,
    "batch_size": 16,
    "lr": 1e-5,
    "weight_decay": 1e-4,
    "num_epochs": 100,
}
from torcheeg.model_selection import KFoldPerSubjectGroupbyTrial
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch.utils.data import DataLoader
from classifier import ClassifierTrainer
k_fold = KFoldPerSubjectGroupbyTrial(
    n_splits=4,
    shuffle=True,
    random_state=42)

training_metrics = []
test_metrics = []

for i, (training_dataset, test_dataset) in enumerate(k_fold.split(dataset)):
    delete_folder_if_exists(target_folder_name='lightning_logs')
    model = SwinTransformer3D(patch_size=(4,3,3),
                                in_chans=1,
                                depths=[2,2,4],
                                num_heads=[3,6,8],
                                window_size=(4,2,2),
                                mlp_ratio=2.,
                                drop_rate=0.1,
                                attn_drop_rate=0.1,
                                drop_path_rate=0.3,
                                num_classes=2
                                )
    trainer = ClassifierTrainer(model=model,
                                num_classes=2,
                                lr=HYPERPARAMETERS['lr'],
                                weight_decay=HYPERPARAMETERS['weight_decay'],
                                metrics=["accuracy"],
                                accelerator="gpu")
    
    training_loader = DataLoader(training_dataset,
                             batch_size=HYPERPARAMETERS['batch_size'],
                             shuffle=True)
    test_loader = DataLoader(test_dataset,
                             batch_size=HYPERPARAMETERS['batch_size'],
                             shuffle=False)
    # 提前停止回调
    early_stopping_callback = EarlyStopping(
        monitor='train_loss',
        patience=30, 
        mode='min',
        verbose=True
    )
    
    trainer.fit(training_loader,
                test_loader,
                max_epochs=HYPERPARAMETERS['num_epochs'],
                callbacks=[early_stopping_callback],
                # enable_progress_bar=True,
                enable_model_summary=False,
                limit_val_batches=0.0)
    # training_result = trainer.test(training_loader,
    #                                enable_progress_bar=True,
    #                                enable_model_summary=True)[0]
    test_result = trainer.test(test_loader,
                               enable_progress_bar=True,
                               enable_model_summary=True)[0]
    # training_metrics.append(training_result["test_accuracy"])
    test_metrics.append(test_result["test_accuracy"])
     
print({
    # "training_metric_avg": np.mean(training_metrics),
    # "training_metric_std": np.std(training_metrics),
    "test_metric_avg": np.mean(test_metrics),
    "test_metric_std": np.std(test_metrics)
})

[2025-02-14 14:14:44] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2025-02-14 14:14:44] INFO (torcheeg/MainThread) 😊 | Please set split_path to .torcheeg/model_selection_1739513684951_Wa2m7 for the next run, if you want to use the same setting for the experiment.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


已删除文件夹: /root/autodl-tmp/.autodl/kinlaw/GANSER/mycode/lightning_logs
Epoch 0: 100%|██████████| 8/8 [00:00<00:00, 24.90it/s, loss=0.708, train_loss=0.645, train_accuracy=0.500]

Metric train_loss improved. New best score: 0.645


Epoch 0: 100%|██████████| 8/8 [00:00<00:00, 24.78it/s, loss=0.708, train_loss=0.645, train_accuracy=0.500]

/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_accuracy', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
[2025-02-14 14:14:45] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.708 train_accuracy: 0.458 



...

Epoch 99: 100%|██████████| 8/8 [00:00<00:00, 26.20it/s, loss=0.0224, train_loss=0.022, train_accuracy=1.000]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /root/autodl-tmp/.autodl/kinlaw/GANSER/mycode/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 102.53it/s]

[2025-02-14 14:18:50] INFO (torcheeg/MainThread) 
[Test] test_loss: 1.271 test_accuracy: 0.625 



Testing DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 86.14it/s] 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy                0.625
        test_loss           1.2707585096359253
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'test_metric_avg': 0.5937499925494194, 'test_metric_std': 0.05962119243443663}


In [4]:
from eegswintransformer import SwinTransformer

HYPERPARAMETERS = {
    "seed": 42,
    "batch_size": 4,
    "lr": 1e-4,
    "weight_decay": 1e-5,
    "num_epochs": 20,
}
from torcheeg.model_selection import KFoldPerSubjectGroupbyTrial
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch.utils.data import DataLoader
from classifier import ClassifierTrainer
k_fold = KFoldPerSubjectGroupbyTrial(
    n_splits=4,
    shuffle=True,
    random_state=42)

training_metrics = []
test_metrics = []

for i, (training_dataset, test_dataset) in enumerate(k_fold.split(dataset)):
    delete_folder_if_exists(target_folder_name='lightning_logs')
    model = SwinTransformer(patch_size=(32,3,3),depths=(2, 2, 2, 2), num_heads=(1, 1, 1, 1))
    trainer = ClassifierTrainer(model=model,
                                num_classes=2,
                                lr=HYPERPARAMETERS['lr'],
                                weight_decay=HYPERPARAMETERS['weight_decay'],
                                metrics=["accuracy"],
                                accelerator="gpu")
    
    training_loader = DataLoader(training_dataset,
                             batch_size=HYPERPARAMETERS['batch_size'],
                             shuffle=True)
    test_loader = DataLoader(test_dataset,
                             batch_size=HYPERPARAMETERS['batch_size'],
                             shuffle=False)
    # 提前停止回调
    early_stopping_callback = EarlyStopping(
        monitor='train_loss',
        patience=10, 
        mode='min',
        verbose=True
    )
    
    trainer.fit(training_loader,
                test_loader,
                max_epochs=HYPERPARAMETERS['num_epochs'],
                callbacks=[early_stopping_callback],
                # enable_progress_bar=True,
                enable_model_summary=False,
                limit_val_batches=0.0)
    # training_result = trainer.test(training_loader,
    #                                enable_progress_bar=True,
    #                                enable_model_summary=True)[0]
    test_result = trainer.test(test_loader,
                               enable_progress_bar=True,
                               enable_model_summary=True)[0]
    # training_metrics.append(training_result["test_accuracy"])
    test_metrics.append(test_result["test_accuracy"])
     
print({
    # "training_metric_avg": np.mean(training_metrics),
    # "training_metric_std": np.std(training_metrics),
    "test_metric_avg": np.mean(test_metrics),
    "test_metric_std": np.std(test_metrics)
})

[2025-02-12 16:56:21] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2025-02-12 16:56:21] INFO (torcheeg/MainThread) 😊 | Please set split_path to .torcheeg/model_selection_1739350581083_W1YOf for the next run, if you want to use the same setting for the experiment.
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarni

Epoch 0: 100%|██████████| 30/30 [00:05<00:00,  5.12it/s, loss=6.65, train_loss=1.400, train_accuracy=0.250]  

Metric train_loss improved. New best score: 1.401


Epoch 0: 100%|██████████| 30/30 [00:05<00:00,  5.12it/s, loss=6.65, train_loss=1.400, train_accuracy=0.250]

/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_accuracy', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
[2025-02-12 16:56:41] INFO (torcheeg/MainThread) 
[Train] train_loss: 8.975 train_accuracy: 0.558 



...

Monitored metric train_loss did not improve in the last 10 records. Best score: 0.000. Signaling Trainer to stop.


Epoch 17: 100%|██████████| 30/30 [00:05<00:00,  5.96it/s, loss=0.000117, train_loss=5.39e-6, train_accuracy=1.000]

[2025-02-12 17:04:04] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.000 train_accuracy: 1.000 



Epoch 17: 100%|██████████| 30/30 [00:05<00:00,  5.96it/s, loss=0.000117, train_loss=5.39e-6, train_accuracy=1.000]


: 

In [5]:
for i, score in enumerate(test_metrics):
    print(i,score)

0 0.44999998807907104
1 0.550000011920929
2 0.6000000238418579
3 0.4749999940395355
4 0.5249999761581421
5 0.625
6 0.699999988079071
7 0.625
